<img style="float:right" src="https://assets-sports.thescore.com/basketball/team/1647/small_logo.png" alt="Miami Emblem" style="width:50px;"/> <h1><b>OSMnx Startercode for Vaskar Raychoudhury's WheelShare research group </b></h1>

## Import and install statements to ensure the code below is functional ##

In [ ]:
!pip install -q geopandas
!pip install -q osmnx
!pip uninstall --yes -q rtree
!sudo apt install -q libspatialindex-dev
!pip install -q rtree

In [ ]:
import pandas as pd
import geopandas as gpd
import networkx as nx
import matplotlib.pyplot as plt
import osmnx as ox
import warnings
warnings.filterwarnings('ignore')
warnings.filterwarnings(action='once')
ox.config(log_console=True, use_cache=True,
          useful_tags_way = ox.settings.useful_tags_way + ['surface_id', 'surface', 'incline', 'wheelchair'],
          useful_tags_node = ox.settings.useful_tags_node + ['wheelchair'])

## Define 'create_graph' for future use => can set our own default peramters for OSMnx graph creation ##

In [ ]:
def create_graph(loc, dist, transport_mode, loc_type="address"):
  """Transport mode = ‘walk’, ‘bike’, ‘drive’, ‘drive_service’, ‘all’, ‘all_private’, ‘none’"""
  if loc_type == "address":
          G = ox.graph_from_address(loc, dist=dist, network_type=transport_mode)
  elif loc_type == "points":
          G = ox.graph_from_point(loc, dist=dist, network_type=transport_mode )
  return G

### Define Point (bentons coordinates) and Tags (Dictionary of filters for pulling data) ###

In [ ]:
point = 39.51101131560681, -84.7335925887448; #Benton Hall Coordinates
tags = {
    'surface' : ['concrete','Asphalt']
}


G = ox.geometries_from_point(point,tags,250)

f, ax = plt.subplots(figsize=(7, 7))
ax.xaxis.set_visible(False)
ax.yaxis.set_visible(False)
G.plot(ax=ax);

# G returns gdf (GeoDataFrame)
G

# f returns plot made from G 
# TODO: convert matplot into oxmnx graph plot.
#f 

<h1><b> Tests </b></h1>

<p>The tests below are to configure the properties needed to display the information we are seeking.<br>Below are some variables needed to run the tests.</p>

In [ ]:
point = 39.51101131560681, -84.7335925887448; 

### Test 1: using the create graph method to plot around an address ###

In [ ]:
#G = create_graph("Oxford, OH", 2000, "walk")
Graph = create_graph("Benton Hall", 150, "walk");
ox.plot_graph(Graph, bgcolor='black')

### Test 2: creating a graph from a point and modifying different perameters passed into plot_graph ###

In [ ]:
#Graph = create_graph(point, 150, "walk", "points")
F = ox.graph_from_point(point, 250, network_type = 'walk', retain_all = True, clean_periphery = True, custom_filter = '["surface"~"concrete"]')
ox.plot_graph(F, bgcolor='#e9ffeb', node_color = '#383d37', edge_color = '#7d8724', node_size = 35)

### Test 3: using test 2's graph and converting it into gdf and back into a graph plot ###

In [ ]:
nodes, edges = ox.graph_to_gdfs(F)
ox.plot_graph(ox.graph_from_gdfs(nodes,edges))

### Test 3.5: same as test 3, however the starting graph was grabbed from an address and had no filter string perameter ### 

In [ ]:
nodes, edges = ox.graph_to_gdfs(Graph)
ox.plot_graph(ox.graph_from_gdfs(nodes,edges))

### Test 4: creates graph around Benton, selects a node at benton to start, and pathes to Armstrong.

In [ ]:
Map = ox.graph_from_point(point,1000)

start = point #predefined as benton hall
end = 39.50802747138995, -84.73306176971025 #location of armstrong studnet center

# Throws errors claiming this method is depricated but supports no further documentation on the topic
start_node = ox.get_nearest_node(Map,start)
end_node = ox.get_nearest_node(Map,end)

# Newer version? Not quite sure what the parameters are though. Output doesn't look accurate.
#start_node = ox.nearest_nodes(Map,start[0],start[1])
#end_node = ox.nearest_nodes(Map,end[0],end[1])

route = nx.shortest_path(Map, start_node, end_node, weight = 'length')
route